In [1]:
import pandas as pd

In [2]:
trx = pd.read_csv("uriage.csv")
trx.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [3]:
customer = pd.read_excel("kokyaku_daicho.xlsx")
customer.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


### 테크닉 . 오류를 확인해보자

In [4]:
# 상품명 공백, 소문자
trx["item_name"].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

In [5]:
# null
trx["item_price"].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

In [6]:
# 이름 사이 공백
customer["고객이름"].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [7]:
# 날짜 형식 불일치
customer["등록일"].head()

0    2018-01-04 00:00:00
1                  42782
2    2018-01-07 00:00:00
3                  42872
4                  43127
Name: 등록일, dtype: object

In [8]:
trx.isnull().sum()

purchase_date      0
item_name          0
item_price       387
customer_name      0
dtype: int64

In [9]:
customer.isnull().sum()

고객이름    0
지역      0
등록일     0
dtype: int64

### 테크닉 . 상품명 오류 수정

In [10]:
trx["purchase_date"] = pd.to_datetime(trx["purchase_date"])
trx["purchase_month"] = trx["purchase_date"].dt.strftime("%Y%m")
trx["purchase_month"].head()

res = pd.pivot_table(trx, index="purchase_month", columns="item_name", aggfunc="size", fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [11]:
#trx["item_name"].unique()
#pd.unique(trx["item_name"])
len(trx["item_name"].unique())

99

In [12]:
trx["item_name"] = trx["item_name"].str.upper()
trx["item_name"] = trx["item_name"].str.replace(" ","")
trx["item_name"].head()

0    상품A
1    상품S
2    상품A
3    상품Z
4    상품A
Name: item_name, dtype: object

In [13]:
len(trx["item_name"].unique())

26

### 테크닉 . 고객명 수정

In [14]:
customer["고객이름"] = customer["고객이름"].str.replace("  ", "")
customer["고객이름"] = customer["고객이름"].str.replace(" ", "")
customer["고객이름"].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

### 테크닉. 금액 결측치 수정

In [15]:
trx.isnull().any(axis=0)
#trx["item_price"].isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

#### loc에 대해서..

In [16]:
city = pd.DataFrame(
    [
        ["Korea","Asia",True],
        ["USA","North America",False],
        ["France","Europe",False]
    ],
    index = ["Busan","Chicago","Paris"],
    columns = ["country","continent","port"]
)
city

,country,continent,port
Busan,Korea,Asia,True
Chicago,USA,North America,False
Paris,France,Europe,False


In [17]:
#loc[조건]
city.loc[city["country"] == "Korea"]

,country,continent,port
Busan,Korea,Asia,True


In [18]:
# loc[조건, 조회할 칼럼 이름]
city.loc[city["country"] == "Korea", "continent"]

Busan    Asia
Name: continent, dtype: object

In [19]:
#Boolean
city.loc[city["port"]]

,country,continent,port
Busan,Korea,Asia,True


In [20]:
#Boolean
city.loc[~city["port"]]

,country,continent,port
Chicago,USA,North America,False
Paris,France,Europe,False


In [21]:
#복합조건 (or / |)
city.loc[(city["country"] == "Korea") | (city["continent"] == "Europe")]

,country,continent,port
Busan,Korea,Asia,True
Paris,France,Europe,False


In [22]:
#복합조건 (and / &)
city.loc[(city["country"] == "Korea") & (city["continent"] == "Europe")]

,country,continent,port


In [23]:
trx["item_price"].isnull()

0       False
1        True
2        True
3       False
4        True
        ...  
2994    False
2995    False
2996     True
2997     True
2998    False
Name: item_price, Length: 2999, dtype: bool

In [24]:
# loc[조건, 조회할 컬럼]
list(trx.loc[trx["item_price"].isnull(),"item_name"])

['상품S',
 '상품A',
 '상품A',
 '상품A',
 '상품A',
 '상품P',
 '상품N',
 '상품W',
 '상품R',
 '상품I',
 '상품W',
 '상품L',
 '상품F',
 '상품N',
 '상품O',
 '상품S',
 '상품B',
 '상품C',
 '상품V',
 '상품Q',
 '상품P',
 '상품I',
 '상품U',
 '상품U',
 '상품C',
 '상품K',
 '상품I',
 '상품T',
 '상품K',
 '상품W',
 '상품X',
 '상품F',
 '상품F',
 '상품Q',
 '상품E',
 '상품N',
 '상품X',
 '상품P',
 '상품O',
 '상품B',
 '상품U',
 '상품U',
 '상품Q',
 '상품S',
 '상품V',
 '상품A',
 '상품M',
 '상품R',
 '상품V',
 '상품T',
 '상품W',
 '상품G',
 '상품G',
 '상품W',
 '상품Q',
 '상품A',
 '상품E',
 '상품S',
 '상품V',
 '상품K',
 '상품N',
 '상품R',
 '상품R',
 '상품W',
 '상품F',
 '상품C',
 '상품J',
 '상품F',
 '상품S',
 '상품B',
 '상품D',
 '상품R',
 '상품W',
 '상품V',
 '상품T',
 '상품D',
 '상품E',
 '상품V',
 '상품X',
 '상품I',
 '상품V',
 '상품V',
 '상품B',
 '상품M',
 '상품F',
 '상품F',
 '상품Q',
 '상품R',
 '상품N',
 '상품W',
 '상품N',
 '상품L',
 '상품A',
 '상품C',
 '상품V',
 '상품L',
 '상품T',
 '상품R',
 '상품X',
 '상품S',
 '상품M',
 '상품C',
 '상품W',
 '상품N',
 '상품E',
 '상품F',
 '상품X',
 '상품K',
 '상품X',
 '상품B',
 '상품W',
 '상품C',
 '상품R',
 '상품A',
 '상품N',
 '상품A',
 '상품C',
 '상품V',
 '상품S',
 '상품Q',
 '상품D',
 '상품S',
 '상품K',
 '상품C',
 '상품R',


In [25]:
# 조건 : (~trx["item_price"].isnull()) & (trx["item_name"] == "상품A")
trx.loc[(trx["item_name"] == "상품A"), "item_price"].max()

100.0

In [26]:
# loc[조건]
trx.loc[(trx["item_price"].isnull()) & (trx["item_name"] == "상품A")]

,purchase_date,item_name,item_price,customer_name,purchase_month
2,2019-05-11 19:42:00,상품A,NaN,김유찬,201905
4,2019-04-22 03:09:00,상품A,NaN,김강현,201904
6,2019-05-18 19:16:00,상품A,NaN,김재준,201905
14,2019-06-11 12:57:00,상품A,NaN,김재호,201906
300,2019-04-06 11:06:00,상품A,NaN,김태인,201904
361,2019-01-21 03:33:00,상품A,NaN,김진우,201901
667,2019-03-16 17:23:00,상품A,NaN,김주원,201903
838,2019-06-24 05:06:00,상품A,NaN,김시호,201906
844,2019-03-15 09:14:00,상품A,NaN,김강현,201903
957,2019-07-19 14:12:00,상품A,NaN,김서윤,201907


In [27]:
price_null = trx["item_price"].isnull()

for item in list(trx.loc[price_null, "item_name"]):
    price = trx.loc[trx["item_name"] == item, "item_price"].max()
    trx["item_price"].loc[(price_null) & (trx["item_name"] == item)] = price

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [28]:
trx["item_price"].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
2994    False
2995    False
2996    False
2997    False
2998    False
Name: item_price, Length: 2999, dtype: bool

In [29]:
trx[["item_price", "item_name"]].head()

,item_price,item_name
0,100.0,상품A
1,1900.0,상품S
2,100.0,상품A
3,2600.0,상품Z
4,100.0,상품A


### 테크닉 날짜 수정

In [45]:
customer["등록일"]

0      2018-01-04 00:00:00
1                    42782
2      2018-01-07 00:00:00
3                    42872
4                    43127
              ...         
195    2017-06-20 00:00:00
196    2018-06-20 00:00:00
197    2017-04-29 00:00:00
198    2019-04-19 00:00:00
199    2019-04-23 00:00:00
Name: 등록일, Length: 200, dtype: object

In [34]:
flg_is_serial = customer["등록일"].astype("str").str.isdigit()
flg_is_serial

0      False
1       True
2      False
3       True
4       True
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 등록일, Length: 200, dtype: bool

In [46]:
flg_is_serial.sum()

22

In [47]:
customer.loc[flg_is_serial, "등록일"].astype("float")

1      42782.0
3      42872.0
4      43127.0
21     42920.0
27     42901.0
47     42741.0
49     42929.0
53     42833.0
76     43188.0
80     43110.0
99     42885.0
114    43254.0
118    43129.0
122    43206.0
139    42880.0
143    42818.0
155    42754.0
172    43181.0
179    42743.0
183    42940.0
186    43294.0
192    43259.0
Name: 등록일, dtype: float64

In [48]:
pd.to_timedelta(customer.loc[flg_is_serial, "등록일"].astype("float"), unit="D")

1     42782 days
3     42872 days
4     43127 days
21    42920 days
27    42901 days
47    42741 days
49    42929 days
53    42833 days
76    43188 days
80    43110 days
99    42885 days
114   43254 days
118   43129 days
122   43206 days
139   42880 days
143   42818 days
155   42754 days
172   43181 days
179   42743 days
183   42940 days
186   43294 days
192   43259 days
Name: 등록일, dtype: timedelta64[ns]

In [49]:
fromSerial = pd.to_timedelta(customer.loc[flg_is_serial, "등록일"].astype("float"), unit="D") + pd.to_datetime("1900/01/01")
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [50]:
fromString = pd.to_datetime(customer.loc[~flg_is_serial, "등록일"])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [51]:
customer["등록일"] = pd.concat([fromSerial, fromString])
customer.head()

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29


In [53]:
customer["등록연월"] = customer['등록일'].dt.strftime('%Y%m')
customer

,고객이름,지역,등록일,등록연월
0,김현성,H시,2018-01-04,201801
1,김도윤,E시,2017-02-18,201702
2,김지한,A시,2018-01-07,201801
3,김하윤,F시,2017-05-19,201705
4,김시온,E시,2018-01-29,201801
...,...,...,...,...
195,김재희,G시,2017-06-20,201706
196,김도영,E시,2018-06-20,201806
197,김이안,F시,2017-04-29,201704
198,김시현,H시,2019-04-19,201904
